In [2]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [3]:
%load_ext autoreload
%autoreload 2

In [7]:
import torch
import transformers
import tqdm
import datetime as dt

In [4]:
# !ls /assets/models/

In [ ]:
# Change this to the model you need to use. Use the above to see the list of models.
model_name_or_path = "/assets/models/mistralai-mistral-instruct-7b-v0.3"
# meta-llama-3.2-3b, meta-llama-3.2-instruct-3b, google-gemma-2-it-2b, google-flan-t5-small
# model_name_or_path = 'sarvamai/sarvam-1'

In [48]:
pwd

'/home/sofia/multilingual/notebooks'

In [49]:
ls /assets/models/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ai4bharat-indictrans2-en-indic-1B/     lmsys-vicuna-7b-v1.5/
allenai-tulu-2-7b/                     lmsys-vicuna-7b-v1.5-16k/
bigscience-bloom-7.1b/                 meta-llama-2-13b/
bigscience-bloomz-3b/                  meta-llama-2-70b/
deepseek-r1-distill-llama-3.1-8b/      meta-llama-2-7b/
deepseek-r1-distill-qwen-2.5-math-7b/  meta-llama-2-chat-13b/
eleutherai-gpt-j-6b/                   meta-llama-2-chat-7b/
eleutherai-pythia-1.4b/                meta-llama-3.1-70b/
eleutherai-pythia-1b/                  meta-llama-3.1-8b/
eleutherai-pythia-2.8b/                meta-llama-3.1-instruct-70b/
eleutherai-pythia-410m/                meta-llama-3.1-instruct-8b/
eleutherai-pythia-6.9b/                meta-llama-3.2-3b/
google-bert-based-uncased/             meta-llama-3.2-instruct-3b/
google-flan-t5-small/                  meta-llama-3.3-instruct-70b/
google-flan-t5-xl/                     meta-llama-3-instruct-8b/
google-gemma-2-it-27b/                 mistralai-mistral-7b-v0.3/
googl

In [50]:
# %pip install protobuf
# %pip install sentencepiece

In [51]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name_or_path)
print("tokenizer loaded")

# LLaMa's tokenizer does not have a valid PAD token, so we need to initialize this as so
# tokenizer.pad_token = tokenizer.eos_token

# For decoder-only models, just to be safe, also do:
tokenizer.padding_side = "left"

tokenizer loaded


In [1]:
tokenizer

NameError: name 'tokenizer' is not defined

In [53]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    # By default, map different parts of the model to available GPU(s).
    device_map="auto",
    # Loading the model in full precision can use a lot of
    # of memory, so we quantize it using reduced precision types.
    torch_dtype='bfloat16'
)

# Best practices
model.eval()

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  4.61it/s]


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

### Generation

More information on using the transformers library and its components can be found here: https://huggingface.co/docs/transformers/llm_tutorial

Specifically, for text generation, the following can be useful:
- https://huggingface.co/docs/transformers/main/en/main_classes/text_generation
- https://huggingface.co/blog/how-to-generate

In [43]:
import datetime as dt

TASK_PROMPT = '''Generate 10 English sentences each for the relations: 
grandmother, grandfather, uncle, aunt, brother-in-law, sister-in-law, cousin, nephew, niece. 
For each relation, generate 10 sentences using the following topics: games, deep talks, questions, exclamations, and other forms of speeches. 
Ensure the sentences include different forms of possessive pronouns (e.g., my, their, his, her) for each of the 9 relations. Avoid using short/colloquial terms for the relations.
Only provide the response as a Python list of strings for all categories. No need to segregate them in any manner.
Sample output: 
<<<[
"My grandmother and I play chess together.", 
...
"Their grandfather and I have deep talks.", 
...
"Due to the heartache, my brother-in-law was not present in the function.",
...
"For the party, my uncle will pick and drop you.",
...
"I have the best sister-in-law in the world!",
...
"I am very grateful to my aunt to take care of my mother when I was not available",
...
... 
] (total 90 sentences: for each relation -> 10 sentences)>>>'''

def generate_response(model, prompt, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt")
    print("tokenized input", dt.datetime.now())

    with torch.inference_mode():
        outputs = model.generate(
            **inputs.to(model.device),
            # temperature=0.7,
            # do_sample=True,
            # top_k=50, 
            # top_p=0.90,
            # num_return_sequences=1,
            # num_beams=3,
            # max_new_tokens=500,
            max_length=100,
            # early_stopping=True,
            # no_repeat_ngram_size=3,
        )
        print("generated output", dt.datetime.now())

        outputs = tokenizer.batch_decode(
            outputs, skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )
        print("decoded output", dt.datetime.now())
    return outputs



In [ ]:
# for output in outputs:
#     print("o/p::", output)
    # print('-' * 50)
    # print()
# save the outputs to a file named : generated_outputs.txt
# open in append mode

def save_outputs(outputs):
    with open('generated_outputs_hin_mf.txt', 'a+') as f:
        for output in outputs:
            f.write(output + '\n')
            # print(output)






In [45]:
def extract_sentences(outputs, prompt):
    response = outputs[0]
    sentences = response[response.find(prompt) + len(prompt):]
    sentences = sentences[sentences.find('{') :sentences.find('}')]  
    print(sentences)
    # sentences = sentences.split(',\n')
    # sentences = [sentence.strip() for sentence in sentences]
    # sentences = [sentence[1:-1] for sentence in sentences]
    # print(sentences, type(sentences))
    return sentences

# extract_sentences(outputs)

In [46]:
# save the sentences to a file named : generated_sentences.txt
def save_sentences(sentences):
    with open('generated_sentences_hin_mf.txt', 'a+') as f:
        # for sentence in sentences:
            f.write(sentences + '\n')


## Prompt template:


In [47]:
prompts=[]
# techniques = examples.keys()
topics=["occupation", " religion", " sport", " politics", "  health", " finance", " education", " farming", " entertainment", "  news", "  daily conversation", "  weather", " technology", " conflicts", " controversials", 
        " international", "  UN", " travel", " tourism", " shopping", " baby care", " valentines", "  soldiers", " prisioners", " soul actions", " nature", " pollution", " bio hazards", " elders", " family", 
        "  social studies", " maths", " literature", "  physics", " chemistry", "  biology", " Indian history ", " civics", " geography", "  computer", " physical education", "  arts and craft", "  food", 
        " clothes", " water shortage", " road blockage", " traffic", " arriving late", " bargaining", " toys", " games", " deep talks", "declarative", "interrogative", "imperative",  "exclamatory", "safe work space", 
        "medical checkups", "self-reliant", "future India"]
topics = [ "legal", "governance",  
  "STEM" ,   "business", "sports",  "culture", "Alarm",
    "Audio",
    "Calendar / Events",  # Added space for better readability
    "Communication",
    "DateTime",
    "Email",
    "Finance",  # Likely, but limited information available
    "General",
    "Home Automation",  # Likely, but limited information available
    "Location",
    "Music",
    "News",
    "Reminders",
    "Social Media",  # Likely, but limited information available
    "Travel",
    "Weather",
    "BookRestaurant",
    "BookFlight",
    "BookHotel",
    "GetDirections",
    "GetPlaceDetails",
    "GetWeather",
    "SearchForInformation",
    "GetNews",
    "PlayMusic",
    "ControlVolume",
    "SetAlarm",
    "CancelAlarm",
    "SetReminder",
    "MakeCall",
    "SendMessage",
    "SendEmail",
    "CheckEmail",
    "GetTime",
    "GetDate",
    "Greeting",
    "Farewell",
    "AskHowAreYou",
    "ThankYou",
    "TellAJoke",
    "TurnOnLights",  # and other home automation actions
    "Social media interactions",  # posting, liking, etc.
    "Financial transactions"  ]

topics=topics[:3]
len(topics)

3

In [48]:
N_samples = 3
tgt_lang = 'Hindi'

In [49]:
def generate_sentences(topics, model, N_samples, tokenizer):
    # for technique in techniques:
        # pick every 10 topics from topics in a loop
    for i in tqdm.tqdm(range(0, len(topics))):
        # print(topics[i:i+10])
        # print("Generate 100 sentences in "+ lang +" with action doer as Male or Female. Differentiate between male and female sentences using the "+ technique +" technique. You can use the topics like "+ ", ".join(topics[i:i+10]) +".Return as a python dictionary.")
        # prompts.append("Generate 10 sentences in "+ lang +" with action doer as Male or Female. Differentiate between male and female sentences using the "+ technique +" technique. You can use the topics like "+ ", ".join(topics[i:i+10]) +".Return as a python dictionary.")
    #     prompt = '''Generate 10 English sentences each for the relations: 
    # grandmother, grandfather, uncle, aunt, brother-in-law, sister-in-law, cousin, nephew, niece. 
    # For each relation, generate 10 sentences using the following topic: ''' + topics[i]+''' 
    # Ensure the sentences include different forms of possessive pronouns (e.g., my, their, his, her) for each of the 9 relations. Avoid using short/colloquial terms for the relations.
    # Only provide the response as a Python list of strings for all categories. No need to segregate them in any manner.
    # Sample output: 
    # <<<[
    # "My grandmother and I play chess together.", 
    # ...
    # "Their grandfather and I have deep talks.", 
    # ...
    # "Due to the heartache, my brother-in-law was not present in the function.",
    # ...
    # "For the party, my uncle will pick and drop you.",
    # ...
    # "I have the best sister-in-law in the world!",
    # ...
    # "I am very grateful to my aunt to take care of my mother when I was not available",
    # ...
    # ... 
    # ] (total 90 sentences: for each relation -> 10 sentences)>>>'''  
    #     prompt = '''Generate '''+str(2*N_samples)+' '+tgt_lang+''' sentences such that subject is male and female individually. Give the gender as key: 0 for Male and 1 for female.
    #     For each gender, generate '''+ str (N_samples)+''' sentences each for following topic: ''' + topics[i]+''' 
    # Only provide the response as a Python list of strings for all categories. No need to segregate them in any manner.
    # Sample output: 
    # <<<{
    # 0: [  
    #             "लेखक नई पुस्तक लिख रहा है।",  
    #             ...
    #         ],  
    #         1: [  
    #             "लेखिका नई पुस्तक लिख रही है।",  
    #             ... 
    #         ]  
    # } (total '''+str(2*N_samples)+''' sentences: for each gender -> '''+ str (N_samples)+''' sentences)>>>''' 
        # prompt = " 'कानून' विषय पर हिंदी वाक्य उत्पन्न करें, जिसमें विषय व्यक्तिगत रूप से महिला हों ।"
        prompt = "Generate hindi sentences on the topic: " + topics[i]+" with the subject as a female."
        print(prompt)
        prompts.append(prompt)

        outputs= generate_response(model, prompt, tokenizer)
        print("outputs done for topic: ", topics[i], i)
        save_outputs(outputs)
        sentences= extract_sentences(outputs, prompt)
        print("sentences extracted for topic:", topics[i], i)
        save_sentences(sentences)

            

In [51]:
models= [  '/assets/models/meta-llama-3.2-instruct-3b',
        #  '/assets/models/google-flan-t5-small', '/assets/models/meta-llama-3.2-3b', '/assets/models/meta-llama-3.2-instruct-3b', 'sarvamai/sarvam-1', '/assets/models/mistralai-mistral-instruct-7b-v0.3'
         ]
 #  ,  ,  ,  
for m in models:
    model = transformers.AutoModelForCausalLM.from_pretrained(
        m,
        # By default, map different parts of the model to available GPU(s).
        device_map="auto",
        # Loading the model in full precision can use a lot of
        # of memory, so we quantize it using reduced precision types.
        torch_dtype='bfloat16'
    )
    tokenizer = transformers.AutoTokenizer.from_pretrained(m)

    # Best practices
    model.eval()
    for i in range(len(topics)):
        generate_sentences(topics, model, N_samples, tokenizer)
 

  0%|          | 0/3 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generate hindi sentences on the topic: legal with the subject as a female.
tokenized input 2025-02-27 12:44:36.262335


 33%|███▎      | 1/3 [03:50<07:41, 230.89s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


generated output 2025-02-27 12:48:27.145824
decoded output 2025-02-27 12:48:27.149987
outputs done for topic:  legal 0

sentences extracted for topic: legal 0
Generate hindi sentences on the topic: governance with the subject as a female.
tokenized input 2025-02-27 12:48:27.155529


 67%|██████▋   | 2/3 [07:41<03:50, 230.60s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

generated output 2025-02-27 12:56:08.182850
decoded output 2025-02-27 12:56:08.184850
outputs done for topic:  STEM 2

sentences extracted for topic: STEM 2


  0%|          | 0/3 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generate hindi sentences on the topic: legal with the subject as a female.
tokenized input 2025-02-27 12:56:08.191545


  0%|          | 0/3 [00:18<?, ?it/s]


KeyboardInterrupt: 

# Google flan

In [75]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")

input_text = '''Generate 6 Hindi sentences such that subject is male and female individually. 
      For each gender, generate 3 sentences each for following topic: legal 
Only provide the response as a Python list of strings for all categories. No need to segregate them in any manner.
'''
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# outputs = model.generate(input_ids, max_new_tokens=100)
# print(tokenizer.decode(outputs[0]))


from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.79it/s]


<pad> Wie alt sind Sie?</s>


For RLHF'd models (LLaMa-3.1 Instruct, etc.), an additional prompt formatting step is needed to ensure that the model is able to generate the desired output. The template is applied using `tokenizer.apply_chat_template` function, and basically adds formatting tokens to your prompt. Use it only with instruction-fine-tuned models.

In [ ]:
# # %pip install jinja2>=3.1.0

# TASK_PROMPT = "Please answer my question. What is the capital of India?"
# TASK_CONVERSATION = [
#     # System Prompt: This is optional, and not all models support this.
#     # But use it if you need explicit instructions to be followed.
#     dict(role='system', content='You are a helpful assistant.'),
#     # Your message (as if on the web interface) goes here.
#     # Past history can be added to this conversation too.
#     dict(role='user', content=TASK_PROMPT)
# ]

# # Format the conversation to a text prompt, using apply chat template.
# conversation_prompt = tokenizer.apply_chat_template(
#     TASK_CONVERSATION,
#     tokenize=False,
#     # Needed to allow the model to start its reply instead of completing yours.
#     add_generation_prompt=True
# )
# # We skip special tokens because the template already adds them. This is an overlooked thing, so be careful.
# inputs = tokenizer(conversation_prompt, return_tensors="pt", add_special_tokens=False)

# # Generation process is the same as before.
# with torch.inference_mode():
#     outputs = model.generate(
#         **inputs.to(model.device),
#         temperature=0.2,
#         do_sample=True,
#         num_return_sequences=2,
#         num_beams=2,
#         max_new_tokens=10
#     )

#     outputs = tokenizer.batch_decode(
#         outputs, skip_special_tokens=True,
#         clean_up_tokenization_spaces=True
#     )

#     for output in outputs:
#         print(output)
#         print('-' * 50)
#         print()

# Indic models

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("sarvamai/sarvam-1")
tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-1")

# Example usage
# text = "कर्नाटक की राजधानी है:"

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.11it/s]


In [ ]:

# text= '''6 हिंदी वाक्य उत्पन्न करें, जिसमें विषय व्यक्तिगत रूप से पुरुष और महिला हों। लिंग को कुंजी के रूप में दें: 0 पुरुष   के लिए और 1 महिला  के लिए।  

# प्रत्येक लिंग के लिए, कानूनी विषय पर 3-3 वाक्य उत्पन्न करें।  

# केवल Python  List के रूप में स्ट्रिंग्स  प्रदान करें। वाक्यों को किसी भी प्रकार से अलग करने की आवश्यकता नहीं है।  

# नमूना:
#     {  
#     0: [  
#                 "वकील अदालत में मामला लड़ रहा है।", 
#                  "लेखक नई पुस्तक लिख रहा है।", 
#                 ...  
#             ],  
#             1: [  
#                 "वकीला अदालत में मामला लड़ रही है।",  
#                  "लेखिका नई पुस्तक लिख रही है।", 
#                 ...  
#             ]  
#     } (कुल 6 वाक्य: प्रत्येक लिंग के लिए 3 वाक्य) '''
text= ''' 'कानून' विषय पर हिंदी वाक्य उत्पन्न करें, जिसमें विषय व्यक्तिगत रूप से महिला हों ।
'''

inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=80,
                          temperature=0.5,
            # do_sample=True,
            # top_k=50, 
            # top_p=0.90,
            # num_return_sequences=1,
            # num_beams=3,
            # max_new_tokens=500,
            # early_stopping=True,
            # no_repeat_ngram_size=3,
              )
for out in outputs:
    result = tokenizer.decode(out)
    print(result)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 'कानून' विषय पर हिंदी वाक्य उत्पन्न करें, जिसमें विषय व्यक्तिगत रूप से महिला हों ।
</s>


In [10]:
# prompt = '''Generate 3 Hindi Sentences regarding lawyer. '''
prompt= ''' [ "वकील अदालत में मामला लड़ रहा है।", "लेखक नई पुस्तक लिख रहा है।", "वकीला अदालत में मामला लड़ रही है।", "लेखिका नई पुस्तक लिख रही है।", '''
inputs = tokenizer(prompt, return_tensors="pt")
print("tokenized input", dt.datetime.now())

with torch.inference_mode():
    outputs = model.generate(
        **inputs.to(model.device),
        # temperature=0.7,
        # do_sample=True,
        # top_k=50, 
        # top_p=0.90,
        # num_return_sequences=1,
        # num_beams=3,
        max_new_tokens=20,
        # early_stopping=True,
        # no_repeat_ngram_size=3,
    )
    print("generated output", dt.datetime.now())

    outputs = tokenizer.batch_decode(
        outputs, skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )
    print("decoded output", dt.datetime.now())



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tokenized input 2025-02-27 11:44:31.371158
generated output 2025-02-27 11:44:36.222296
decoded output 2025-02-27 11:44:36.224013


In [11]:
outputs

['[ "वकील अदालत में मामला लड़ रहा है।", "लेखक नई पुस्तक लिख रहा है।", "वकीला अदालत में मामला लड़ रही है।", "लेखिका नई पुस्तक लिख रही है।", [/INST]\n ## चरण 1: वकील अदालत में मामला लड़ रहा है, यह वाक्यांश एक क्रिया है']

# Airavata

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
def create_prompt_with_chat_format(messages, bos="<s>", eos="</s>", add_bos=True):
    formatted_text = ""
    for message in messages:
        if message["role"] == "system":
            formatted_text += "<|system|>\n" + message["content"] + "\n"
        elif message["role"] == "user":
            formatted_text += "<|user|>\n" + message["content"] + "\n"
        elif message["role"] == "assistant":
            formatted_text += "<|assistant|>\n" + message["content"].strip() + eos + "\n"
        else:
            raise ValueError(
                "Tulu chat template only supports 'system', 'user' and 'assistant' roles. Invalid role: {}.".format(
                    message["role"]
                )
            )
    formatted_text += "<|assistant|>\n"
    formatted_text = bos + formatted_text if add_bos else formatted_text
    return formatted_text


def inference(input_prompts, model, tokenizer):
    input_prompts = [
        create_prompt_with_chat_format([{"role": "user", "content": input_prompt}], add_bos=False)
        for input_prompt in input_prompts
    ]

    encodings = tokenizer(input_prompts, padding=True, return_tensors="pt")
    encodings = encodings.to(device)

    with torch.inference_mode():
        outputs = model.generate(encodings.input_ids, do_sample=False, max_new_tokens=50)

    output_texts = tokenizer.batch_decode(outputs.detach(), skip_special_tokens=True)

    input_prompts = [
        tokenizer.decode(tokenizer.encode(input_prompt), skip_special_tokens=True) for input_prompt in input_prompts
    ]
    output_texts = [output_text[len(input_prompt) :] for input_prompt, output_text in zip(input_prompts, output_texts)]
    return output_texts

In [ ]:

model_name = "ai4bharat/Airavata"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)

In [25]:


input_prompts = [
    # "मैं अपने समय प्रबंधन कौशल को कैसे सुधार सकता हूँ? मुझे पांच बिंदु बताएं।",
    # "मैं अपने समय प्रबंधन कौशल को कैसे सुधार सकता हूँ? मुझे पांच बिंदु बताएं और उनका वर्णन करें।",
    "'कानून' विषय पर हिंदी वाक्य उत्पन्न करें, जिसमें विषय व्यक्तिगत रूप से महिला हों ।"
]
outputs = inference(input_prompts, model, tokenizer)
print(outputs)

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  5.95it/s]
/home/sofia/anaconda3/envs/syn_data_gen/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/sofia/anaconda3/envs/syn_data_gen/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


KeyboardInterrupt: 

# English sentences

  -  Generate english sentences using I or 1st person as neutral
 -   translate these senetnces into Hindi using Sarvam API  translate or mayura-v1 model: for 2 times using 1st as speaker-gender male and 2nd as female
-    collect the translated sentences.

In [ ]:
model_path = 'assets/models/meta-llama-3.2-instruct-3b'
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_path,
    # By default, map different parts of the model to available GPU(s).
    device_map="auto",
    # Loading the model in full precision can use a lot of
    # of memory, so we quantize it using reduced precision types.
    torch_dtype='bfloat16'
)

tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)



In [57]:
topics = ['occupation']
N_samples = 5
tgt_lang = 'Hindi'


In [58]:
outputs_list = []
for topic in topics:
    prompt = f"Generate {N_samples} sentences such that first person is used as the subject. The topic is {topic}.Return as a python list of sentences."
    print(model, prompt)
    # outputs = model.generate(prompt, tokenizer)
    inputs = tokenizer(prompt, return_tensors="pt")
    print("tokenized input", dt.datetime.now())

    with torch.inference_mode():
        outputs = model.generate(
            **inputs.to(model.device),
            # temperature=0.7,
            # do_sample=True,
            # top_k=50, 
            # top_p=0.90,
            # num_return_sequences=1,
            # num_beams=3,
            # max_new_tokens=500,
            max_length=50,
            # early_stopping=True,
            # no_repeat_ngram_size=3,
        )
        print("generated output", dt.datetime.now())

        outputs = tokenizer.batch_decode(
            outputs, skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )
        print("decoded output", dt.datetime.now())
    # return outputs
    print(outputs)

    # save the outputs to a file named : syn_eng_sents.txt
    # open in append mode
    with open('syn_eng_sents.txt', 'a') as f:
        for output in outputs:
            f.write(output + '\n')

    outputs_list.append(outputs)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

KeyboardInterrupt: 

In [59]:
outputs_list

[]

In [ ]:
# for each sentence, make the payload for sarva api using both male and female, translate to hindi and save the hindi sentences in a file